In [1]:
import igraph as ig
import time
import json
import requests
from random import shuffle
import plotly
import plotly.plotly as py
from plotly.graph_objs import *

In [2]:
#Get the API key and username from the https://plot.ly
username = ""
api_key = ""
plotly.tools.set_credentials_file(username= username, api_key=api_key)

In [3]:
#load file
network = json.load(open('network.json'))

In [4]:
print(len(network))

689


In [5]:
#1 is visible - public
#0 is invisible - private
all_users = {}
labels = []
group = []
all_users['throughshutter'] = 0
group.append(1)
labels.append('throughshutter')
first = network['throughshutter']
number = 1
for a,b,c in first:
    all_users[str(a)] = number
    labels.append(a)
    if c == False:
        group.append(1)
    else:
        group.append(0)
    number = number+1

In [6]:
for a,b,c in first:
    child = network[str(a)]
    for d,e,f in child:
        if str(d) in all_users:
            continue
        else:
            all_users[str(d)] = number
            labels.append(d)
            if f == False:
                group.append(1)
            else:
                group.append(0)
            number = number+1

In [7]:
#print(all_users['throughshutter'])

In [8]:
edges=[]
for a,b in network.items():
    for item in b:
        edges.append((all_users[a],all_users[item[0]]))
edges = edges[:10000]
shuffle(edges)
edges = edges[:10000]
#print(len(edges))
#print(edges)

In [9]:
edges=[]
root = list(network.keys())[0]
childs = list(network.values())[0]
network.pop(root)
connections = {}
connections['throughshutter'] = len(childs)
for item in childs:
    edges.append((all_users[root],all_users[item[0]]))

In [10]:
for a,b in network.items():
    common = []
    for item1 in b:
        for item2 in childs:
            if item1[0] == item2[0]:
                #print(item1,item2)
                common.append(item1)
    #print(a,common)
    for item in common:
        edges.append((all_users[a],all_users[item[0]]))
    if len(common) != 0:
        connections[a] = len(common)

In [11]:
print(len(edges))

11702


In [12]:
label_temp = labels
labels = []
for item in label_temp:
    try:
        labels.append(item+" has "+str(connections[item]) + " common connections")
    except:
        labels.append(item)

In [13]:
G=ig.Graph(edges, directed=False)
layt=G.layout('kk', dim=3) #else use drl

In [14]:
N = len(layt)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]  

In [15]:
trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,  
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=6, 
                             color=group, 
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

In [16]:
axis=dict(showbackground=False,
          showline=False,  
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )

In [17]:
layout = Layout(
         title="Instagram Network Visualization by Ayush Joshi", 
         width=1000,
         height=1000,
         showlegend=False,
         scene=Scene(  
         xaxis=XAxis(axis),
         yaxis=YAxis(axis), 
         zaxis=ZAxis(axis), 
        ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False, 
            text="Data source: <a href='https://www.instagram.com'>[1] Instagram</a>",
            xref='paper',     
            yref='paper',     
            x=0,  
            y=0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=Font(
            size=14 
            )     
            )
        ]),    )

In [18]:
data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)
#plot_url = py.plot(fig) #get url